In [1]:
pip install -r requirements.txt || true

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests as rq
import numpy as np
from datetime import datetime, timedelta
import os

# Демонстрация того, как будет происходить процесс совмещения на двух днях

In [3]:
# Ссылка на данные
url1 = "https://iss.moex.com/iss/analyticalproducts/futoi/securities/si.json?from=2023-11-07&till=2023-11-08&table_type=full"
url2 = "https://iss.moex.com/iss/analyticalproducts/futoi/securities/si.json?from=2023-11-06&till=2023-11-07&table_type=full"

In [4]:
# Отправляем GET-запрос по указанной ссылке
response = rq.get(url1)

# Проверяем статус ответа
if response.status_code == 200:
    # Загружаем данные из ответа в переменную data в случае успеха
    data = response.json()
else:
    print("Ошибка при получении данных")
    

columns = response.json()["futoi"]['columns']

data = response.json()["futoi"]["data"]

df1 = pd.DataFrame(data, columns=columns)


# Отправляем GET-запрос по указанной ссылке
response = rq.get(url2)

# Проверяем статус ответа
if response.status_code == 200:
    # Загружаем данные из ответа в переменную data в случае успеха
    data = response.json()
else:
    print("Ошибка при получении данных")


columns = response.json()["futoi"]['columns']

data = response.json()["futoi"]["data"]

df2 = pd.DataFrame(data, columns=columns)

In [5]:
print(df1.sort_values(by=['ticker', 'tradedate','tradetime']).reset_index().drop('index', axis=1))
print(len(np.array(df1['ticker'])))

     sess_id  seqnum   tradedate tradetime ticker clgroup     pos  pos_long  \
0       6922      59  2023-11-07  09:05:00     Si     FIZ -155138    439394   
1       6922      59  2023-11-07  09:05:00     Si     YUR  155138   3192186   
2       6922      60  2023-11-07  09:10:00     Si     FIZ -155331    439692   
3       6922      60  2023-11-07  09:10:00     Si     YUR  155331   3192080   
4       6922      61  2023-11-07  09:15:00     Si     FIZ -155165    439792   
..       ...     ...         ...       ...    ...     ...     ...       ...   
695     6924      56  2023-11-08  23:40:00     Si     YUR  140369   3221244   
696     6924      57  2023-11-08  23:45:00     Si     FIZ -140319    456549   
697     6924      57  2023-11-08  23:45:00     Si     YUR  140325   3221244   
698     6924      58  2023-11-08  23:50:00     Si     FIZ -140215    455996   
699     6924      58  2023-11-08  23:50:00     Si     YUR  140215   3221177   

     pos_short  pos_long_num  pos_short_num        

In [6]:
df2.sort_values(by=['ticker', 'tradedate', 'tradetime']).reset_index().drop('index', axis=1)

,sess_id,seqnum,tradedate,tradetime,ticker,clgroup,pos,pos_long,pos_short,pos_long_num,pos_short_num,systime
0,6921,59,2023-11-06,09:05:00,Si,FIZ,-152370,438179,-590549,7646,12891,2023-11-06 09:05:03
1,6921,59,2023-11-06,09:05:00,Si,YUR,152370,3193000,-3040630,234,188,2023-11-06 09:05:03
2,6921,60,2023-11-06,09:10:00,Si,FIZ,-152566,437884,-590450,7660,12881,2023-11-06 09:10:05
3,6921,60,2023-11-06,09:10:00,Si,YUR,152566,3192851,-3040285,237,185,2023-11-06 09:10:05
4,6921,61,2023-11-06,09:15:00,Si,FIZ,-152331,437356,-589687,7675,12905,2023-11-06 09:15:05
...,...,...,...,...,...,...,...,...,...,...,...,...
695,6923,56,2023-11-07,23:40:00,Si,YUR,117102,3193534,-3076432,241,183,2023-11-07 23:40:06
696,6923,57,2023-11-07,23:45:00,Si,FIZ,-117198,460148,-577346,7857,12383,2023-11-07 23:45:06
697,6923,57,2023-11-07,23:45:00,Si,YUR,117198,3193535,-3076337,242,182,2023-11-07 23:45:06
698,6923,58,2023-11-07,23:50:00,Si,FIZ,-117280,459912,-577192,7844,12367,2023-11-07 23:50:06


In [7]:
print(len(np.array(df2['ticker'])))

700


In [8]:
frames = [df1, df2]

result = pd.concat(frames).sort_values(by=['ticker', 'tradedate', 'tradetime']).reset_index().drop('index', axis=1)
filename = "aboba_date.csv"
result.to_csv(filename, sep=',', index=False, encoding='utf-8')

print(result)

print(len(np.array(result['ticker'])))

      sess_id  seqnum   tradedate tradetime ticker clgroup     pos  pos_long  \
0        6921      59  2023-11-06  09:05:00     Si     FIZ -152370    438179   
1        6921      59  2023-11-06  09:05:00     Si     YUR  152370   3193000   
2        6921      60  2023-11-06  09:10:00     Si     FIZ -152566    437884   
3        6921      60  2023-11-06  09:10:00     Si     YUR  152566   3192851   
4        6921      61  2023-11-06  09:15:00     Si     FIZ -152331    437356   
...       ...     ...         ...       ...    ...     ...     ...       ...   
1395     6924      56  2023-11-08  23:40:00     Si     YUR  140369   3221244   
1396     6924      57  2023-11-08  23:45:00     Si     FIZ -140319    456549   
1397     6924      57  2023-11-08  23:45:00     Si     YUR  140325   3221244   
1398     6924      58  2023-11-08  23:50:00     Si     FIZ -140215    455996   
1399     6924      58  2023-11-08  23:50:00     Si     YUR  140215   3221177   

      pos_short  pos_long_num  pos_shor

Далее будет пробегать по массиву дат и таким образом сформируем всю таблицу

Ниже я задаю массив всех дат

# Парсинг данных о позиций

Сначала мы создаём массив, который содержить все даты, за которые мы хотим получить данные

In [9]:
start_date = datetime(2020, 5, 8)
end_date = datetime.now()

current_date = start_date
array_date = []
while current_date <= end_date:
    array_date += [current_date.strftime("%Y-%m-%d")]
    current_date += timedelta(days=1)

Тут мы формируем массив из дней, а затем, если сервер работает хорошо, то возвращаем данные за весь промежуток от 5 августа 2020 до сегодняшнего дня. 

Если сервер перестаёт отвечать, тогда полученные данные мы собираем в файл и при повторном запуске программа проверить наши файлы и начнёт с последнего записанного дня.

В качестве отладки печатается дата дня, который мы сейчас получаем. 

In [10]:
def add_new_day(date_today: str, ticker_name: str):
    date_yesterday = (datetime.strptime(date_today, '%Y-%m-%d') - timedelta(days=1)).strftime("%Y-%m-%d")
    url = f"https://iss.moex.com/iss/analyticalproducts/futoi/securities/{ticker_name}.json?from={date_today}&till={date_today}&table_type=full"

    while True:
        try:
            # Ваш код, который может вызвать ошибку
            response = rq.get(url)
            # break  # Если ошибки не возникло, выходим из цикла
        except Exception as e:
            # Обработка других исключений
            print(f"Произошла ошибка: {e}")
        else:
            break

    # Проверяем статус ответа
    if response.status_code != 200:
        print("Ошибка при получении данных")
        add_new_day(date_today)

    df_for_add = pd.DataFrame(response.json()["futoi"]["data"], columns=response.json()["futoi"]['columns'])
    df_for_add.sort_values(by=['ticker', 'tradedate','tradetime']).reset_index().drop('index', axis=1)
   
    return df_for_add

# ticker_names=['si', 'br', 'ri', 'eu', 'mx', 'sr', 'ox', 'gz', 'yn', 'fv', 'vi', 'lk']
ticker_names=['sr', 'gz', 'lk', 'vb', 'rn', 'mn', 'af', 'al', 'sn', 'yn', 'tt', 'nm', 'hy', 'me', 'fv', 'gk', 'mg', 'ml']

for ticker_name in ticker_names:
    if os.path.exists(f"{ticker_name}_full_date.csv"):
        df = pd.read_csv(f"{ticker_name}_full_date.csv")
        last_valid_date_index = df['tradedate'].last_valid_index()
        last_valid_date_value  = df['tradedate'][last_valid_date_index]
        date_tomorrow = (datetime.strptime(last_valid_date_value, '%Y-%m-%d') + timedelta(days=1)).strftime("%Y-%m-%d")
        print(date_tomorrow)
        if(not date_tomorrow in array_date):
            pass
        else:
            df_array = [df]
            for date in array_date[array_date.index(date_tomorrow)+1:]:
                df_array += [add_new_day(date, ticker_name)]
                print(date)
            df = pd.concat(df_array).sort_values(by=['ticker', 'tradedate', 'tradetime']).reset_index().drop('index', axis=1)
            print(df)
            filename = f"{ticker_name}_full_date.csv"
            df.to_csv(filename, sep=',', index=False, encoding='utf-8')
    else:
        print(f"Файл '{ticker_name}_full_date.csv' не существует.")
        df_array = []
        for date in array_date:
            df_array += [add_new_day(date, ticker_name)]
            print(date)
        df = pd.concat(df_array).sort_values(by=['ticker', 'tradedate', 'tradetime']).reset_index().drop('index', axis=1)
        print(df)
        filename = f"{ticker_name}_full_date.csv"
        df.to_csv(filename, sep=',', index=False, encoding='utf-8')

    

Файл 'sr_full_date.csv' не существует.
2020-05-08
2020-05-09
2020-05-10
2020-05-11
2020-05-12
2020-05-13
2020-05-14
2020-05-15
2020-05-16
2020-05-17
2020-05-18
2020-05-19
2020-05-20
2020-05-21
2020-05-22
2020-05-23
2020-05-24
2020-05-25
2020-05-26
2020-05-27
2020-05-28
2020-05-29
2020-05-30
2020-05-31
2020-06-01
2020-06-02
2020-06-03
2020-06-04
2020-06-05
2020-06-06
2020-06-07
2020-06-08
2020-06-09
2020-06-10
2020-06-11
2020-06-12
2020-06-13
2020-06-14
2020-06-15
2020-06-16
2020-06-17
2020-06-18
2020-06-19
2020-06-20
2020-06-21
2020-06-22
2020-06-23
2020-06-24
2020-06-25
2020-06-26
2020-06-27
2020-06-28
2020-06-29
2020-06-30
2020-07-01
2020-07-02
2020-07-03
2020-07-04
2020-07-05
2020-07-06
2020-07-07
2020-07-08
2020-07-09
2020-07-10
2020-07-11
2020-07-12
2020-07-13
2020-07-14
2020-07-15
2020-07-16
2020-07-17
2020-07-18
2020-07-19
2020-07-20
2020-07-21
2020-07-22
2020-07-23
2020-07-24
2020-07-25
2020-07-26
2020-07-27
2020-07-28
2020-07-29
2020-07-30
2020-07-31
2020-08-01
2020-08-02
2020

KeyboardInterrupt: 

Удаление повторяющихся данных (в силу того, как МосБиржа возвращала данные (в выходные дни вместо пустого запроса она возвращает последнюю цену за последний рабочий день, то есть 14*20 одинаковых записей, в силу чего эта ячейка позволяла сэкономить, примерно, 300 МБ))

In [ ]:
# ticker_names=['si', 'br', 'ri', 'eu', 'mx', 'sr', 'ox', 'gz', 'yn', 'fv', 'vi', 'lk']

for ticker_name in ticker_names:
    your_file = f"{ticker_name}_full_date.csv" 
# Загрузка данных
    df = pd.read_csv(your_file)

# Удаление повторяющихся строк
    unique_rows = df.drop_duplicates()

# Сохранение результата
    unique_rows.to_csv(your_file, index=False)

Итоговые данные не идеальны, поэтому мы их будем обрабатывать в других данных.